In [ ]:
import pandas as pd

class PureUnityFilterCorrected:
    def __init__(self):
        # Trainer scores
        self.trainer_scores = {
            'aidan o\'brien': 4.0, 'a p o\'brien': 4.0, 'joseph patrick o\'brien': 3.8,
            'donnacha o\'brien': 3.5, 'dermot weld': 3.2, 'jessica harrington': 2.8,
            'william haggas': 3.5, 'john gosden': 3.5, 'j & t gosden': 3.5,
            'charlie appleby': 3.2, 'sir michael stoute': 3.0, 'roger varian': 2.6,
            'andrew balding': 2.4, 'ralph beckett': 2.2,
            'tim easterby': 2.8, 'richard fahey': 2.6, 'david o\'meara': 2.2,
            'michael dods': 2.0, 'kevin ryan': 1.8, 'john quinn': 1.6
        }
        # Jockey scores
        self.jockey_scores = {
            'ryan moore': 3.0, 'william buick': 2.8, 'frankie dettori': 2.8,
            'james doyle': 2.4, 'tom marquand': 2.4, 'oisin murphy': 2.6,
            'jim crowley': 2.2, 'silvestre de sousa': 2.2, 'rossa ryan': 2.0,
            'daniel tudhope': 2.4, 'colin keane': 2.4, 'seamie heffernan': 2.8,
            'wayne lordan': 2.2
        }

    # --- Helpers ---
    def parse_form_corrected(self, form_str):
        if not form_str: return []
        positions = []
        for ch in form_str:
            if ch.isdigit():
                pos = int(ch); positions.append(10 if pos == 0 else pos)
            elif ch.upper() in 'FURPD':
                positions.append(15)
        return positions

    def calculate_corrected_rel(self, form, or_rating, age):
        if not form: return 1.0
        weights = [1.0,1.2,1.5,1.8,2.2,2.8,3.2,3.5]
        recent = form[-8:]
        score=0
        for i,pos in enumerate(recent):
            if i>=len(weights): break
            w=weights[i]
            if pos==1: points=8.0*w
            elif pos==2: points=5.5*w
            elif pos==3: points=4.0*w
            elif pos<=6: points=2.0*w
            elif pos<=10: points=0.5*w
            else: points=0
            score+=points
        extended=form[-12:]
        wins=sum(1 for p in extended if p==1)
        places=sum(1 for p in extended if p<=3)
        frames=sum(1 for p in extended if p<=6)
        consistency=(wins*3+places*2+frames)/len(extended) if extended else 0
        momentum=0.0
        if len(form)>=6:
            early=form[-6:-3]; recent3=form[-3:]
            if early and recent3:
                imp=(sum(early)/len(early))-(sum(recent3)/len(recent3))
                if imp>=3: momentum=2
                elif imp>=1.5: momentum=1
                elif imp>=0.5: momentum=0.5
                elif imp<=-3: momentum=-2
                elif imp<=-1.5: momentum=-1
                elif imp<=-0.5: momentum=-0.5
        base_rel=min(4.0,max(1.0,(score/25)+consistency+momentum))
        if age==3: base_rel*=1.15
        elif age==4: base_rel*=1.10
        elif age>=8: base_rel*=0.90
        if or_rating>=85: base_rel*=0.95
        elif or_rating<=55: base_rel*=1.05
        return round(max(1.0,min(4.0,base_rel)),2)

    def calculate_pure_map(self, stall, field_size, distance, track):
        if stall<=field_size*0.3: draw_score=0.8
        elif stall>=field_size*0.7: draw_score=-0.6
        else: draw_score=0.1
        if field_size>=14: draw_score*=1.3
        elif field_size<=8: draw_score*=0.7
        return round(draw_score,1)

    def calculate_pure_csi(self, trainer, jockey, form):
        csi=0
        trainer_key=trainer.lower()
        for t,s in self.trainer_scores.items():
            if t in trainer_key: csi+=s; break
        jockey_key=jockey.lower()
        for j,s in self.jockey_scores.items():
            if j in jockey_key: csi+=s; break
        return round(min(csi,10.0),1)

    def calculate_pure_tpi(self, or_rating, cd_markers, age, class_level):
        tpi=0
        if or_rating>=100:tpi+=5
        elif or_rating>=90:tpi+=4
        elif or_rating>=80:tpi+=3
        elif or_rating>=70:tpi+=2
        elif or_rating>=60:tpi+=1
        if cd_markers:
            c=cd_markers.upper()
            if 'CD'in c: tpi+=3
            elif c.count('C')+c.count('D')>=2: tpi+=2
            elif 'C'in c or 'D'in c: tpi+=1
        if 4<=age<=6: tpi+=1
        if 'group'in class_level.lower(): tpi+=2
        elif 'listed'in class_level.lower(): tpi+=1
        return min(tpi,12)

    # --- Main Process with EW Guardrails + Top 2 only ---
    def process_race(self, race_data):
        horses = race_data['horses']
        field_size = len(horses)

        # EW guardrails
        if field_size < 7:
            return {
                'qualifies': False,
                'reason': f'No Bet — only {field_size} runners (<7)',
                'selections': []
            }
        if field_size > 15:
            return {
                'qualifies': False,
                'reason': f'No Bet — {field_size} runners (>15)',
                'selections': []
            }

        # Place terms
        places = 2 if 7 <= field_size <= 11 else 3

        processed = []
        for h in horses:
            form = self.parse_form_corrected(h.get('form',''))
            rel = self.calculate_corrected_rel(form, h.get('or',60), h.get('age',5))
            map_score = self.calculate_pure_map(
                h.get('stall',1), field_size,
                race_data['distance'], race_data.get('track','newbury')
            )
            csi = self.calculate_pure_csi(h.get('trainer',''), h.get('jockey',''), form)
            tpi = self.calculate_pure_tpi(
                h.get('or',60), h.get('cd',''),
                h.get('age',5), race_data['class']
            )
            primary = rel + map_score + csi
            total = primary + tpi
            processed.append({
                'name': h['name'], 'stall': h['stall'],
                'rel': rel, 'map': map_score, 'csi': csi, 'tpi': tpi,
                'primary': round(primary,1), 'total': round(total,1)
            })

        processed.sort(key=lambda x: (-x['primary'], -x['tpi'], -x['total']))

        # Only return Top 2
        return {
            'qualifies': True,
            'places': places,
            'field_size': field_size,
            'selections': processed[:2]
        }
